# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/qTEA1/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298457818342                   -0.85    5.1         
  2   -8.300206954736       -2.76       -1.26    1.1    106ms
  3   -8.300435066246       -3.64       -1.89    3.1    140ms
  4   -8.300461201789       -4.58       -2.78    2.1    180ms
  5   -8.300464316921       -5.51       -3.11    3.1    156ms
  6   -8.300464488427       -6.77       -3.30    3.9    146ms
  7   -8.300464563373       -7.13       -3.45    1.5    116ms
  8   -8.300464606989       -7.36       -3.61   11.0    299ms
  9   -8.300464632157       -7.60       -3.78    1.5    152ms
 10   -8.300464638717       -8.18       -3.91    2.0    182ms
 11   -8.300464643522       -8.32       -4.23    4.1    203ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64184097892                   -0.70    7.0         
  2   -16.64604022280       -2.38       -1.14    1.0    263ms
  3   -16.67920004731       -1.48       -1.86    2.1    290ms
  4   -16.67927735245       -4.11       -2.66    5.1    410ms
  5   -16.67928600893       -5.06       -3.00    4.0    414ms
  6   -16.67928618846       -6.75       -3.30    1.9    285ms
  7   -16.67928621575       -7.56       -3.74    1.6    269ms
  8   -16.67928622115       -8.27       -4.49    2.6    323ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32718169820                   -0.56    6.8         
  2   -33.33449761929       -2.14       -1.00    1.2    1.06s
  3   -33.33597166907       -2.83       -1.73    5.0    1.40s
  4   -33.33616830863       -3.71       -2.64    4.5    1.29s
  5   -33.33689691634       -3.14       -2.46    7.2    1.75s
  6   -33.33691765818       -4.68       -2.60    1.5    1.01s
  7   -33.33694315589       -4.59       -3.46    2.5    1.18s
  8   -33.33694374779       -6.23       -4.00    6.1    1.72s
  9   -33.33694378174       -7.47       -4.61    2.8    1.25s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298196836104                   -0.85    4.9         
  2   -8.300214097953       -2.70       -1.59    1.0   82.8ms
  3   -8.300383111053       -3.77       -2.34    3.4    124ms
  4   -8.300323965935   +   -4.23       -2.18    2.4    150ms
  5   -8.300463904174       -3.85       -3.48    1.0   85.2ms
  6   -8.300464536074       -6.20       -3.72   13.8    308ms
  7   -8.300464634963       -7.00       -4.10    2.1    115ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32519025777                   -0.56    6.6         
  2   -33.26548211933   +   -1.22       -1.24    1.1    867ms
  3   +17.84136917942   +    1.71       -0.21    8.0    2.18s
  4   -33.28594066925        1.71       -1.56    6.6    2.05s
  5   -33.13062106309   +   -0.81       -1.26    4.6    1.43s
  6   -31.27359320140   +    0.27       -0.91    4.1    1.36s
  7   -33.29718481182        0.31       -1.70    4.0    1.34s
  8   -33.33073102978       -1.47       -2.07    2.2    1.02s
  9   -33.33242560028       -2.77       -1.94    2.8    1.30s
 10   -33.33672205127       -2.37       -2.70    2.0    1.10s
 11   -33.33692455167       -3.69       -2.94    3.5    1.34s
 12   -33.33692590617       -5.87      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.